# Ensemble Model
Putting together an ensemble of the average wordvec model and a model using the other features.

# Reading the data

In [28]:
import pandas as pd
train = pd.read_csv('train_cleaned.csv')
test = pd.read_csv('test_cleaned.csv')

In [29]:
wordvec_train = pd.read_pickle('train_wordvec.pickle')
wordvec_test = pd.read_pickle('test_wordvec.pickle')

In [30]:
train = train.merge(wordvec_train, on=['id'])
test = test.merge(wordvec_test, on=['id'])
train.head()

,id,keyword,location,text,target,cleaned_text,wordvec,keyword_wordvec
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this earthquake Ma...,"[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[-0.26623327, 0.05843069, -0.1404636, -0.05265..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,Forest fire near La Ronge Sask. Canada,"[-0.025449565, 0.031005142, -0.15566371, -0.23...","[-0.025449565, 0.031005142, -0.15566371, -0.23..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,"[0.0059339865, 0.016337818, -0.105279535, -0.0...","[0.0059339865, 0.016337818, -0.105279535, -0.0..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...,"[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[-0.18147185, 0.20731743, 0.014147284, -0.2182..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby Alaska as s...,"[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[-0.06394094, -0.01423019, 0.0063574947, 0.071..."


In [31]:
train_tree = pd.read_csv('train_metafeatures_tree.csv')
test_tree = pd.read_csv('test_metafeatures_tree.csv')
train_normalized = pd.read_csv('train_metafeatures_normalized.csv')
test_normalized = pd.read_csv('test_metafeatures_normalized.csv')
train_normalized.head()

,id,num_char_n,num_words_n,num_hash_n,num_mention_n,num_url_n,has_location,geocoded,longitude_n,latitude_n,target
0,1,0.433121,0.419355,0.076923,0.0,0.0,False,False,0.0,0.0,1
1,4,0.242038,0.225806,0.000000,0.0,0.0,False,False,0.0,0.0,1
2,5,0.847134,0.709677,0.000000,0.0,0.0,False,False,0.0,0.0,1
3,6,0.356688,0.225806,0.076923,0.0,0.0,False,False,0.0,0.0,1
4,7,0.541401,0.516129,0.153846,0.0,0.0,False,False,0.0,0.0,1


In [32]:
train_tree = train_tree.merge(train, on=['id','target'])
train_normalized = train_normalized.merge(train, on=['id','target'])
test_tree = test_tree.merge(test, on=['id'])
test_normalized = test_normalized.merge(test, on = ['id'])
train_normalized.head()

,id,num_char_n,num_words_n,num_hash_n,num_mention_n,num_url_n,has_location,geocoded,longitude_n,latitude_n,target,keyword,location,text,cleaned_text,wordvec,keyword_wordvec
0,1,0.433121,0.419355,0.076923,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,Our Deeds are the Reason of this earthquake Ma...,"[-0.26623327, 0.05843069, -0.1404636, -0.05265...","[-0.26623327, 0.05843069, -0.1404636, -0.05265..."
1,4,0.242038,0.225806,0.000000,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Forest fire near La Ronge Sask. Canada,Forest fire near La Ronge Sask. Canada,"[-0.025449565, 0.031005142, -0.15566371, -0.23...","[-0.025449565, 0.031005142, -0.15566371, -0.23..."
2,5,0.847134,0.709677,0.000000,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,All residents asked to 'shelter in place' are ...,All residents asked to 'shelter in place' are ...,"[0.0059339865, 0.016337818, -0.105279535, -0.0...","[0.0059339865, 0.016337818, -0.105279535, -0.0..."
3,6,0.356688,0.225806,0.076923,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders in ...,"[-0.18147185, 0.20731743, 0.014147284, -0.2182...","[-0.18147185, 0.20731743, 0.014147284, -0.2182..."
4,7,0.541401,0.516129,0.153846,0.0,0.0,False,False,0.0,0.0,1,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,Just got sent this photo from Ruby Alaska as s...,"[-0.06394094, -0.01423019, 0.0063574947, 0.071...","[-0.06394094, -0.01423019, 0.0063574947, 0.071..."


## Putting all data in one SVM model
Puts metafeatures and features in one big SVM model.

In [33]:
train_normalized.columns

Index(['id', 'num_char_n', 'num_words_n', 'num_hash_n', 'num_mention_n',
       'num_url_n', 'has_location', 'geocoded', 'longitude_n', 'latitude_n',
       'target', 'keyword', 'location', 'text', 'cleaned_text', 'wordvec',
       'keyword_wordvec'],
      dtype='object')

In [40]:
import numpy
def get_X(df, col_wv, cols_ext):
    X = []
    for index, row in df.iterrows():
        x = row[col_wv]
        x = numpy.append(row[cols_ext].values, x)
        X.append(x)        
    return X

In [43]:
cols_ext = ['num_char_n', 'num_words_n', 'num_hash_n', 'num_mention_n',
       'num_url_n', 'has_location', 'geocoded', 'longitude_n', 'latitude_n']
X = get_X(train_normalized, 'wordvec',cols_ext)
y = train_normalized['target']

In [44]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
svm = SVC(kernel="rbf")
params = {'C': [0.05, 0.1, 0.2, 0.5, 0.75, 1, 1.5, 2, 4]}
clf = GridSearchCV(svm, params, scoring="f1", verbose=1, n_jobs=-2, cv=5)

In [45]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X, y, cv=cv, return_train_score=True, scoring='f1')
scores

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  2.5min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  3.0min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  4.2min finished


Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  4.2min finished


{'fit_time': array([158.19189024, 189.40911245, 242.12806582, 265.95231915,
        270.01944232]),
 'score_time': array([2.1467371 , 2.26172423, 3.16707468, 3.3029592 , 3.25709343]),
 'test_score': array([0.76399027, 0.74728941, 0.7749577 , 0.76101695, 0.77564637]),
 'train_score': array([0.82279274, 0.79297365, 0.81854249, 0.78786602, 0.79731149])}

In [46]:
scores['test_score'].mean()

0.7645801390849052

slight improvement?

In [47]:
def prepare_submission(model, X, y, X_test, name):
    fit = model.fit(X,y)
    pred = model.predict(X_test)
    submission = pd.DataFrame({"id":test['id'], "target":pred})
    submission.to_csv(name+'.csv', index=False)
    return fit

In [48]:
X_test = get_X(test_normalized, 'wordvec',cols_ext)

In [49]:
fit = prepare_submission(clf, X, y, X_test, 'avg_wordvec_meta_svm')

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=-2)]: Done  45 out of  45 | elapsed:  3.8min finished


In [50]:
fit.best_params_

{'C': 0.5}

Not a public score improvement